# Quick start

*EPAM Syngen* is an unsupervised tabular data generation tool based on a variational autoencoder (VAE). 
It supports common tabular datatypes (floats, integers, datetime, text, categorical, binary) and can generate linked tables that sharing keys using the simple statistical approach. 
The SDK exposes simple programmatic entry points for training, inference, report generation, loading and saving data in supported formats - *CSV*, *Avro* and *Excel* format. The data should be located locally and be in UTF-8 encoding.

This notebook demonstrates the SDK usage. Install the package and then you can call the main SDK class `Syngen` to run training, inference or generation of reports, and the class `DataIO` to load and save the data in supported formats.

Python *3.10* or *3.11* is required to run the library. The library is tested on Linux and Windows operating systems.

# Installation

Please, install the library *syngen* (from Pypi):

In [2]:
!pip install  --index-url https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ --use-pep517 --no-cache-dir syngen==0.10.32rc5

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 6.0 MB/s  0:00:00m eta 0:00:01
  Attempting uninstall: syngen
    Found existing installation: syngen 0.10.32rc4
    Uninstalling syngen-0.10.32rc4:
      Successfully uninstalled syngen-0.10.32rc4


# Class initialization

```python
Syngen(
    metadata_path: Optional[str] = None,  # use a metadata file in the '.yaml', '.yml' format for a training or an inference of one or multiple tables to centralize all parameters in one place
    table_name: Optional[str] = None,     # required for a single-table training or inference process; an arbitrary string used to name the directories where artifacts are stored
    source: Optional[str] = None,         # optional for a single-table training or inference process; a path to the file that you want to use as a reference
    loader: Optional[Callable[[str], pd.DataFrame]] # optional for a single-table training or inference process; a callback function that returns the sample of the original data
)
```
### Attributes description:

- **`metadata_path`** *(Optional[str], default: None)*: a path to a metadata file in *'.yaml'* or *'.yml'* format, used during a training or inference processes to centralize all parameters for one or multiple tables in one place.
- **`table_name`** *(Optional[str], default: None)*: a required parameter for a training or inference processes of a single table; an arbitrary string used to name the directories where artifacts are stored.
- **`source`** *(Optional[str], default: None)*: an optional parameter for a single-table training or inference process; a path to the file that you want to use as a reference.
- **`loader`** *(Optional[str], default: None)*: an optional parameter for a single-table training or inference process; a callback function that returns the sample of the original data.

***Note***: You can provide the information about required attributes in one of three ways:
1. Use `metadata_path` to define the a metadata of one or multiple tables and their relationships in one place.
2. Using `table_name` and `source`: For a single-table training or inference, provide both `table_name` and `source` if the original data is supplied directly from a `source`.
3. Using `table_name` and `loader`: For a single-table training or inference, provide both `table_name` and `loader` if the original data is supplied via a callback function (`loader`).

In [3]:
# The example of the initialization of the instance of the class 'Syngen' for the single-table training or inference process
# by providing the `source` and 'table_name'parameters
from syngen.sdk import Syngen


launcher_for_single_table_with_source = Syngen(
    source="../examples/example-data/housing.csv", 
    table_name="housing"
)

2026-01-02 14:53:27.815234: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-02 14:53:27.815271: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-02 14:53:27.816165: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# The example of the initialization of the instance of the class 'Syngen' for the single-table training or inference process 
# by providing the `loader` and 'table_name' parameters
import os
import pandas as pd

from syngen.sdk import Syngen



def get_dataframe(table_name: str) -> pd.DataFrame:
    """
    The demonstration callback function to load a CSV file 
    and get a pandas DataFrame from the 'example-data' directory.
    
    Parameters
    ----------
    table_name : str
        The name of the table (CSV file without extension) to load
    
    Returns
    -------
    pd.DataFrame
        The loaded data as a pandas DataFrame
    """
    path_to_example_data = f"../examples/example-data/{table_name}.csv"
    
    if not os.path.exists(path_to_example_data):
        raise FileNotFoundError(f"The CSV file '{table_name}.csv' does not exist")
    
    return pd.read_csv(path_to_example_data)



launcher_for_single_table_with_loader = Syngen(
    loader=get_dataframe, 
    table_name="housing"
)

In [5]:
# The example of the initialization of the instance of the class 'Syngen' for multiple tables
from syngen.sdk import Syngen


launcher_for_multiple_tables = Syngen(
    metadata_path="../examples/example-metadata/housing_metadata.yaml"
)

# Launch training

You can start a training process using the SDK entrypoint `Syngen().train(...)`. This will train a model and save the model artifacts to a disk in the directory *'./model_artifacts'*. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
train(
    self,
    epochs: int = 10,                     # a number of training epochs
    drop_null: bool = False,              # whether to drop rows with at least one missing value
    row_limit: Optional[int] = None,      # a number of rows to train over
    reports: Union[str, Tuple[str], List[str]] = "none",  # report types: "none", "accuracy", "sample", "metrics_only", "all"
    log_level: Literal["TRACE", "DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"] = "INFO", # a logging level
    batch_size: int = 32,                 # a training batch size
    fernet_key: Optional[str] = None      # a name of the environment variable containing the Fernet key for secure storage of the data subset
)
```

### Parameters description:

- **`epochs`** *(int, default: 10)*: A number of training epochs. Must be ≥ 1. Since the early stopping mechanism is implemented the bigger value of epochs is the better.

- **`drop_null`** *(bool, default: False)*: Whether to drop rows containing at least one missing value before training. When `False`, missing values are handled during the training process.

- **`row_limit`** *(Optional[int], default: None)*: A maximum number of rows to use for training. If specified and less than the total rows, a random subset of the specified size will be selected. Useful for testing or working with large datasets.

- **`reports`** *(Union[str, Tuple[str], List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing synthetic data (same size as original) with original dataset to estimate the quality of training process
  - `"sample"` - generates a sample report showing distribution comparisons between the original data and the subset of this data
  - `"metrics_only"` - outputs metrics to stdout without generation of an accuracy report
  - `"all"` - generates both accuracy and sample reports

  List example: `["accuracy", "sample"]` to generate multiple report types

  *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: A logging level for the training process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`batch_size`** *(int, default: 32)*: A training batch size. Must be ≥ 1. Splits training into batches to optimize memory usage. Smaller batches use less RAM but may increase training time.

- **`fernet_key`** *(Optional[str], default: None)*: A name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is encrypted on a disk (stored in the `.dat` format). If not provided, data is stored unencrypted in the `.pkl` format. **Important**: The same key must be used during an inference and a report generation to decrypt the data.


*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [6]:
# The example of a training the single table on the provided data fetching it via the 'source' parameter


launcher_for_single_table_with_source.train(
    epochs=5,
    drop_null=False,
    row_limit=1000, 
    batch_size=100,
    reports="all",
    log_level="DEBUG"
)

2026-01-02 14:53:48.400 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-02 14:53:48.401 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/resources/housing/' was removed
2026-01-02 14:53:48.405 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2026-01-02 14:53:48.405 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-02 14:53:48.405 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-02 14:53:48.405 | INFO     | syngen.train:launch_train:104 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate inform

32/32 [==============================] - 0s 797us/step


2026-01-02 14:53:53.889 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-02 14:53:53.890 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-02 14:53:56.316 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-02 14:53:56.368 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing/vae/checkpoints
2026-01-02 14:53:56.368 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-02 14:53:56.369 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-02 14:53:56.369 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing' was completed
2026-01-02 14:53:56.370 | INFO     | syngen.ml.worker.worker:_infer_table:520 - Infer process of the table - 'housing' has started


32/32 [==============================] - 0s 1ms/step


2026-01-02 14:53:56.783 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-02 14:53:56.784 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=1000, run_parallel=False, batch_size=1000, random_seed=1
2026-01-02 14:53:56.784 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-02 14:53:56.784 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 1
2026-01-02 14:53:56.784 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 14:53:56.785 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 6935.86it/s]
2026-01-02 14:53:57.018 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%)

In [7]:
# The example of training the single table on the provided data fetching it via the 'loader' parameter


launcher_for_single_table_with_loader.train(
    epochs=5,
    drop_null=False,
    row_limit=1000, 
    batch_size=100, 
    reports="all",
    log_level="DEBUG"
)

2026-01-02 14:54:53.631 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-02 14:54:53.635 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/resources/housing/' was removed
2026-01-02 14:54:53.635 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2026-01-02 14:54:53.636 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-02 14:54:53.636 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-02 14:54:53.636 | INFO     | syngen.train:launch_train:104 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate inform

32/32 [==============================] - 0s 867us/step


2026-01-02 14:54:58.583 | INFO     | syngen.ml.vae.models.model:fit_sampler:179 - Fit sampler
2026-01-02 14:54:58.591 | INFO     | syngen.ml.vae.models.model:fit_sampler:183 - Start encoding
2026-01-02 14:54:58.741 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-02 14:54:58.741 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-02 14:55:00.939 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-02 14:55:00.979 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing/vae/checkpoints
2026-01-02 14:55:00.980 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-02 14:55:00.980 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-02 14:55:00.980 | INFO     | syngen.ml.strategies.strategies:run:153 - Train

32/32 [==============================] - 0s 1ms/step


2026-01-02 14:55:01.387 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-02 14:55:01.387 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=1000, run_parallel=False, batch_size=1000, random_seed=1
2026-01-02 14:55:01.388 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-02 14:55:01.388 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 1
2026-01-02 14:55:01.388 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 14:55:01.388 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.
Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 7599.63it/s]
2026-01-02 14:55:01.625 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%)

In [8]:
launcher_for_single_table_with_loader.execution_artifacts

{'housing': {'losses_path': 'model_artifacts/system_store/losses/losses-housing-2026-01-02-14-54-53-655124.csv',
  'path_to_input_data': 'model_artifacts/tmp_store/housing/input_data_housing.pkl',
  'generated_reports': {'sample_report': 'model_artifacts/resources/housing/reports/sample-report-2026_01_02_14_55_04_954935.html',
   'accuracy_report': 'model_artifacts/resources/housing/reports/accuracy-report-2026_01_02_14_55_28_405311.html'}}}

In [9]:
# The example of training of multiple tables with relationships


launcher_for_multiple_tables.train(log_level="DEBUG")

2026-01-02 14:57:28.885 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-02 14:57:28.890 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2026-01-02 14:57:28.890 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-02 14:57:28.890 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:create_tracker:69 - MLFlow server is either unreachable or not set up, therefore the tracking will not be performed
2026-01-02 14:57:28.890 | INFO     | syngen.train:launch_train:104 - The training process will be executed according to the information mentioned in 'train_settings' in the metadata file. If appropriate information is absent from the metadata file, then the values of parameters sent through CLI will be used. Otherwise, the values of parameters will be defaulted.
2026-01-02 14:57:28.8

25/25 [==============================] - 0s 750us/step


2026-01-02 14:57:41.868 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-02 14:57:41.868 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-02 14:57:42.366 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-02 14:57:42.399 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-properties/vae/checkpoints
2026-01-02 14:57:42.400 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-02 14:57:42.400 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-02 14:57:42.400 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing_properties' was completed
2026-01-02 14:57:42.408 | INFO     | syngen.ml.processors.processors:_preprocess_data:121 - As the parameter 'drop_null' set t

57/57 [==============================] - 0s 606us/step


2026-01-02 14:57:55.504 | INFO     | syngen.ml.vae.models.model:fit_sampler:187 - Creating BayesianGaussianMixture
2026-01-02 14:57:55.504 | INFO     | syngen.ml.vae.models.model:fit_sampler:189 - Fitting BayesianGaussianMixture
2026-01-02 14:57:56.831 | INFO     | syngen.ml.vae.models.model:fit_sampler:191 - Finished fitting BayesianGaussianMixture
2026-01-02 14:57:56.860 | INFO     | syngen.ml.vae.wrappers.wrappers:save_state:545 - Saved VAE state in model_artifacts/resources/housing-conditions/vae/checkpoints
2026-01-02 14:57:56.861 | INFO     | syngen.ml.handlers.handlers:__fit_model:191 - Finished VAE training
2026-01-02 14:57:56.861 | INFO     | syngen.ml.handlers.handlers:handle:143 - No columns to train kde over found
2026-01-02 14:57:56.861 | INFO     | syngen.ml.strategies.strategies:run:153 - Training of the table - 'housing_conditions' was completed
2026-01-02 14:57:56.864 | INFO     | syngen.ml.worker.worker:_infer_table:520 - Infer process of the table - 'housing_properti

...

2026-01-02 14:57:57.196 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2026-01-02 14:57:57.196 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=790, run_parallel=False, batch_size=790, random_seed=1
2026-01-02 14:57:57.196 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-02 14:57:57.196 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2026-01-02 14:57:57.197 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 14:57:57.197 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 7220.89it/s]
2026-01-02 14:57:57.376 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table 

32/32 [==============================] - 0s 803us/step


2026-01-02 14:57:57.579 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-conditions/vae/checkpoints
2026-01-02 14:57:57.580 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=1799, run_parallel=False, batch_size=1000, random_seed=1
2026-01-02 14:57:57.580 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 2 batch(es)
2026-01-02 14:57:57.580 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 1 of 2
2026-01-02 14:57:57.580 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 14:57:57.580 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4173.44it/s]

25/25 [==============================] - 0s 992us/step



2026-01-02 14:57:57.713 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_conditions'. Generating the batch 2 of 2
2026-01-02 14:57:57.713 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 14:57:57.713 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_conditions' started.
Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 4875.21it/s]
2026-01-02 14:57:57.796 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2026-01-02 14:57:57.814 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2026-01-02 14:57:57.815 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-02 14:5

The *'execution_artifacts'* attribute of the **Syngen** class provides information about the generated artifacts during the training process:

In [10]:
from pprint import pprint

pprint(launcher_for_multiple_tables.execution_artifacts)

{'housing_conditions': {'generated_reports': {'accuracy_report': 'model_artifacts/resources/housing-conditions/reports/accuracy-report-2026_01_02_14_58_02_496768.html',
                                              'sample_report': 'model_artifacts/resources/housing-conditions/reports/sample-report-2026_01_02_14_57_58_529476.html'},
                        'losses_path': 'model_artifacts/system_store/losses/losses-housing-conditions-2026-01-02-14-57-42-411068.csv',
                        'path_to_input_data': 'model_artifacts/tmp_store/housing-conditions/input_data_housing-conditions.pkl'},
 'housing_properties': {'generated_reports': {'accuracy_report': 'model_artifacts/resources/housing-properties/reports/accuracy-report-2026_01_02_14_58_11_293503.html',
                                              'sample_report': 'model_artifacts/resources/housing-properties/reports/sample-report-2026_01_02_14_58_03_597866.html'},
                        'losses_path': 'model_artifacts/system_sto

# Launch generation of synthetic data

You can start an inference process using the SDK entrypoint `Syngen().infer(...)`. The SDK mirrors the CLI options so you can pass the same parameters programmatically. Below is a complete description of all available parameters:

```python
infer(
    self,
    size: int = 100,                      # the desired number of rows to generate
    run_parallel: bool = False,           # whether to use multiprocessing (feasible for tables > 50000 rows)
    batch_size: Optional[int] = None,     # an inference batch size
    random_seed: Optional[int] = None,    # if specified, generates a reproducible result
    reports: Union[str, List[str]] = "none",  # report types: "none", "accuracy", "metrics_only", "all"
    log_level: str = "INFO",              # a logging level
    fernet_key: Optional[str] = None      # a name of the environment variable containing the Fernet key for decrypting the data subset
)
```
### Parameters description:

- **`size`** *(int, default: 100)*: The desired number of synthetic rows to generate. Must be ≥ 1.

- **`run_parallel`** *(bool, default: False)*: Whether to use multiprocessing for data generation. Set to `True` to enable parallel processing, which is recommended and feasible for generating large tables (> 50000 rows).

- **`batch_size`** *(Optional[int], default: None)*: The inference batch size. Must be ≥ 1. If specified, the generation is split into batches to optimize memory usage and save RAM.

- **`random_seed`** *(Optional[int], default: None)*: A random seed for reproducible generation. Must be ≥ 0.

- **`reports`** *(Union[str, Tuple[str], List[str]], default: "none")*: Controls generation of quality reports. Accepts single string or list of strings:
  - `"none"` - no reports generated (default)
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data patterns to verify quality of a generated data
  - `"metrics_only"` - outputs metrics to stdout without generating an accuracy report
  - `"all"` - generates an accuracy report (same as `"accuracy"`)

  List example: `["accuracy", "metrics_only"]` to generate multiple report types

  *Note*: Report generation may require significant time for large generated tables (>10,000 rows)

- **`log_level`** *(str, default: "INFO")*: A logging level for the inference process. Accepted values: `"TRACE"`, `"DEBUG"`, `"INFO"`, `"WARNING"`, `"ERROR"`, `"CRITICAL"`.

- **`fernet_key`** *(Optional[str], default: None)*: A name of the environment variable containing a 44-character URL-safe base64-encoded Fernet key. When provided, the data subset is decrypted for a report generation. **Important**: The same key used during a training must be used during a report generation to successfully decrypt the data.


*Note:* For full documentation, metadata file format, and additional details, please refer to [README.md](../README.md)

In [11]:
# The example of inference for the single table

launcher_for_single_table_with_source.infer(
    size=100000,
    run_parallel=False,
    batch_size=5000,
    random_seed=42,
    reports="all"
)

2026-01-02 15:05:01.275 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/merged_infer_housing.csv' was removed
2026-01-02 15:05:01.276 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/infer_message.success' was removed
2026-01-02 15:05:01.276 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:172 - As the destination path wasn't specified for the table - 'housing', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-02 15:05:01.278 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2026-01-02 15:05:01.278 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-02 15:05:01.278 | WARNI

  1/157 [..............................] - ETA: 19s

2026-01-02 15:05:01.704 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-02 15:05:01.705 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 20 batch(es)
2026-01-02 15:05:01.705 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 20
2026-01-02 15:05:01.705 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 15:05:01.705 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3436.93it/s]

...

Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3594.37it/s]

 97/157 [=================>............] - ETA: 0s


2026-01-02 15:05:09.636 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-02 15:05:09.636 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 20 of 20
2026-01-02 15:05:09.636 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 15:05:09.636 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3531.10it/s]
2026-01-02 15:05:10.038 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-02 15:05:10.365 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-02 15:05:10.366 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-02 15:05:10.366 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing' has started
2026-01-02 15:05:11.708 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8172
Generating bivariate distributions...: 100%|██████████| 45/45 [00:39<00:00,  1.14it/s]
2026-01-02 15:05:55.372 | I

In [12]:
launcher_for_single_table_with_source.execution_artifacts

{'housing': {'path_to_input_data': 'model_artifacts/tmp_store/housing/input_data_housing.pkl',
  'path_to_generated_data': 'model_artifacts/tmp_store/housing/merged_infer_housing.csv',
  'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing/reports/accuracy-report-2026_01_02_15_05_58_731659.html'}}}

In [13]:
# The example of inference for the single table

launcher_for_single_table_with_loader.infer(
    size=100000,
    run_parallel=False,
    batch_size=5000,
    random_seed=42,
    reports="all"
)

2026-01-02 15:07:09.883 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/merged_infer_housing.csv' was removed
2026-01-02 15:07:09.884 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing/infer_message.success' was removed
2026-01-02 15:07:09.884 | WARNING  | syngen.ml.config.validation:_check_existence_of_destination:172 - As the destination path wasn't specified for the table - 'housing', the synthetic data will be stored at the default path - './model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-02 15:07:09.885 | INFO     | syngen.ml.config.validation:_collect_errors:435 - The validation of the metadata has been passed successfully
2026-01-02 15:07:09.885 | WARNING  | syngen.ml.mlflow_tracker.mlflow_tracker:check_mlflow_server:32 - MLFlow server URL not provided
2026-01-02 15:07:09.885 | WARNI

  1/157 [..............................] - ETA: 20s

2026-01-02 15:07:10.307 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing/vae/checkpoints
2026-01-02 15:07:10.308 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 20 batch(es)
2026-01-02 15:07:10.308 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 1 of 20
2026-01-02 15:07:10.308 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 15:07:10.308 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 1ms/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 2967.60it/s]

...

Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3552.85it/s]

103/157 [==================>...........] - ETA: 0s


2026-01-02 15:07:18.052 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-02 15:07:18.052 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing'. Generating the batch 20 of 20
2026-01-02 15:07:18.052 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 15:07:18.053 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing' started.


157/157 [==============================] - 0s 998us/step


Generation of the data...: 100%|██████████| 11/11 [00:00<00:00, 3466.63it/s]
2026-01-02 15:07:18.455 | INFO     | syngen.ml.vae.wrappers.wrappers:_restore_nan_values:135 - Column 'total_bedrooms' has 0 (0.0%) empty values generated
2026-01-02 15:07:18.791 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing/merged_infer_housing.csv'
2026-01-02 15:07:18.791 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-02 15:07:18.791 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing' has started
2026-01-02 15:07:20.155 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8172
Generating bivariate distributions...: 100%|██████████| 45/45 [00:40<00:00,  1.11it/s]
2026-01-02 15:08:04.750 | I

In [14]:
launcher_for_single_table_with_loader.execution_artifacts

{'housing': {'path_to_input_data': 'model_artifacts/tmp_store/housing/input_data_housing.pkl',
  'path_to_generated_data': 'model_artifacts/tmp_store/housing/merged_infer_housing.csv',
  'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing/reports/accuracy-report-2026_01_02_15_08_08_039010.html'}}}

In [15]:
# The example of inference of multiple tables with relationships


launcher_for_multiple_tables.infer(log_level="DEBUG")

2026-01-02 15:10:07.731 | DEBUG    | syngen.ml.validation_schema.validation_schema:validate_schema:348 - The schema of the metadata is valid
2026-01-02 15:10:07.731 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/merged_infer_housing-properties.csv' was removed
2026-01-02 15:10:07.731 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-properties/infer_message.success' was removed
2026-01-02 15:10:07.732 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv' was removed
2026-01-02 15:10:07.732 | INFO     | syngen.ml.worker.worker:_remove_existed_artifact:104 - The artifacts located in the path - 'model_artifacts/tmp_store/housing-conditions/infer_message.success' was rem

3/3 [==============================] - 0s 2ms/step


2026-01-02 15:10:08.050 | INFO     | syngen.ml.vae.wrappers.wrappers:load_state:554 - Loaded VAE state from model_artifacts/resources/housing-properties/vae/checkpoints
2026-01-02 15:10:08.050 | DEBUG    | syngen.ml.handlers.handlers:handle:490 - Infer model with parameters: size=90, run_parallel=False, batch_size=90, random_seed=10, reports - 'accuracy'
2026-01-02 15:10:08.050 | INFO     | syngen.ml.handlers.handlers:handle:491 - Total of 1 batch(es)
2026-01-02 15:10:08.050 | INFO     | syngen.ml.handlers.handlers:handle:503 - Data synthesis for the table - 'housing_properties'. Generating the batch 1 of 1
2026-01-02 15:10:08.050 | INFO     | syngen.ml.handlers.handlers:run:348 - Start data synthesis
2026-01-02 15:10:08.051 | INFO     | syngen.ml.handlers.handlers:run_separate:323 - VAE generation for 'housing_properties' started.
Generation of the data...: 100%|██████████| 7/7 [00:00<00:00, 11697.26it/s]
2026-01-02 15:10:08.184 | INFO     | syngen.ml.strategies.strategies:run:236 - S

3/3 [==============================] - 0s 2ms/step


Generation of the data...: 100%|██████████| 3/3 [00:00<00:00, 6410.04it/s]
2026-01-02 15:10:08.480 | INFO     | syngen.ml.handlers.handlers:generate_keys:428 - The 'households' assigned as a foreign_key feature
2026-01-02 15:10:08.484 | INFO     | syngen.ml.strategies.strategies:run:236 - Synthesis of the table - 'housing_conditions' was completed. Synthetic data saved in 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv'
2026-01-02 15:10:08.484 | WARNING  | syngen.ml.reporters.reporters:generate_report:241 - The report(s) generation might be time-consuming
2026-01-02 15:10:08.485 | INFO     | syngen.ml.reporters.reporters:_log_and_update_progress:287 - The calculation of accuracy metrics for the table - 'housing_conditions' has started
2026-01-02 15:10:08.653 | INFO     | syngen.ml.metrics.accuracy_test.accuracy_test:_fetch_metrics:193 - Median accuracy is 0.8211
Generating bivariate distributions...: 100%|██████████| 3/3 [00:01<00:00,  1.78it/s]
2026-0

The *'execution_artifacts'* attribute of the **Syngen** class provides information about the generated artifacts during the inference process:

In [16]:
pprint(launcher_for_multiple_tables.execution_artifacts)

{'housing_conditions': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-conditions/reports/accuracy-report-2026_01_02_15_10_11_927093.html'},
                        'path_to_generated_data': 'model_artifacts/tmp_store/housing-conditions/merged_infer_housing-conditions.csv',
                        'path_to_input_data': 'model_artifacts/tmp_store/housing-conditions/input_data_housing-conditions.pkl'},
 'housing_properties': {'generated_reports': {'accuracy_report': 'model_artifacts/tmp_store/housing-properties/reports/accuracy-report-2026_01_02_15_10_19_496679.html'},
                        'path_to_generated_data': 'model_artifacts/tmp_store/housing-properties/merged_infer_housing-properties.csv',
                        'path_to_input_data': 'model_artifacts/tmp_store/housing-properties/input_data_housing-properties.pkl'}}


# Data security: Using Fernet Key for encryption

In the current implementation, a sample of the original data is stored on a disk during a training process. To ensure data security and protect sensitive information, you can use a **Fernet key** to encrypt this data.

## What is a Fernet Key?

A Fernet key is a 44-character URL-safe base64-encoded string used for symmetric encryption. When provided, the data subset is encrypted on a disk (stored in the `.dat` format instead of unencrypted `.pkl` format).

## How to Generate a Fernet Key

You can generate a Fernet key using the following code:

In [ ]:
# Generate a Fernet key
from cryptography.fernet import Fernet

fernet_key = Fernet.generate_key().decode("utf-8")

## Setting the Fernet Key as an environment variable

After generating the key, you need to store it as an environment variable. This can be done in your terminal or programmatically in Python.

### Option 1: Set in Terminal (Linux/macOS)

```bash
export MY_FERNET_KEY='your_generated_fernet_key_here'
```

### Option 2: Set in Terminal (Windows)

```cmd
set MY_FERNET_KEY=your_generated_fernet_key_here
```

### Option 3: Set programmatically in Python

```python
import os
os.environ['MY_FERNET_KEY'] = 'your_generated_fernet_key_here'
```

## Using the Fernet Key in a training

When training with encryption, pass the name of the environment variable (not the Fernet key itself) to the `fernet_key` parameter:

In [ ]:
# The example: the training with the Fernet key encryption

import os
from syngen.sdk import Syngen

# Step 1: Set the Fernet key as an environment variable
os.environ["MY_FERNET_KEY"] = fernet_key  # Using the key generated above

# Step 2: Train with encryption enabled

launcher_for_encrypted_data = Syngen(
    source="../examples/example-data/housing.csv", 
    table_name="housing_encrypted"
)

launcher_for_encrypted_data.train(
    epochs=5,
    row_limit=1000, 
    batch_size=32, 
    fernet_key="MY_FERNET_KEY"  # Pass the environment variable name, not the key itself
)

## Using the Fernet Key in an inference

**Important**: When generating synthetic data during an inference process, you must use the **same Fernet key** that was used during a training process. This allows the system to decrypt the stored data subset for a report generation.

If the Fernet key is not provided or doesn't match the Fernet key used in the training process, the inference process will fail when trying to access the encrypted data.

In [ ]:
# Example: Inference with Fernet key decryption
# The environment variable 'MY_FERNET_KEY' is already set from the training step

# Inference with the same Fernet key
launcher_for_encrypted_data.infer(
    size=100000,
    batch_size=5000, 
    random_seed=42,
    reports="all",
    fernet_key="MY_FERNET_KEY"  # Must use the same key as in a training
)

In [ ]:
launcher_for_encrypted_data.execution_artifacts

## Using the Fernet Key with the metadata file

You can also specify the Fernet key in the metadata file for both training and inference:

```yaml
global:
  encryption:
    fernet_key: MY_FERNET_KEY  # Name of the environment variable

TABLE_NAME:
  train_settings:
    source: "./data/table.csv"
  
  infer_settings:
    size: 100
  
  # You can also specify per-table encryption
  encryption:
    fernet_key: MY_FERNET_KEY
```

Then use it in your code:

```python
# Training with the metadata file and the Fernet key
Syngen(metadata_path="path/to/metadata.yaml").train()

# Inference with the metadata file and the Fernet key
Syngen(metadata_path="path/to/metadata.yaml").infer()
```

## Important security notes

⚠️ **Critical security considerations:**

1. **Store the key securely**: Never hardcode the Fernet key directly in your code or commit it to version control systems.
2. **Key recovery is impossible**: If you lose the Fernet key, encrypted data cannot be recovered
3. **Same key required**: Always use the same Fernet key for a training, an inference and a report generation
4. **Environment variable**: Use an environment variable to store the Fernet key securely
5. **Key length**: The Fernet key must be exactly 44 characters (URL-safe base64-encoded)
6. **Production environments**: In production, use secure secret management services (AWS Secrets Manager, Azure Key Vault, HashiCorp Vault, etc.)

## What happens without a Fernet key?

If you don't provide a `fernet_key` parameter:
- Data subset is stored **unencrypted** in the `.pkl` format
- No decryption is needed during the inference or the report generation
- Suitable for non-sensitive data or development environments

With a `fernet_key`:
- Data subset is stored **encrypted** in the `.dat` format
- Decryption is required during the inference or the report generation using the same Fernet key
- Recommended for sensitive or production data

# Generate quality reports separately

Sometimes you may want to generate quality reports separately after training or/and inference has already been completed to evaluate the quality of the input data or the generated data. The SDK provides the `Syngen().generate_quality_reports(...)` method that allows you to generate quality reports for a table using existing artifacts without re-running the training or/and inference processes.

This method is useful when:
- You completed training/inference without quality reports (with `reports="none"`)
- You want to generate additional report types later
- You want to separate the computation-intensive training/inference from report generation


```python
generate_quality_reports(
    self,
    table_name: str,                        # required: the name of the table to generate quality reports for
    reports: Union[str, Tuple[str], List[str]], # required: report types to generate
    fernet_key: Optional[str] = None        # optional: a Fernet key for decrypting encrypted data
)
```

### Parameters description:

- **`table_name`** *(str, required)*: The name of the table to generate reports for.

- **`reports`** *(Union[str, Tuple[str], List[str]], required)*: Controls which quality reports to generate. Accepts single string or list of strings:
  - `"accuracy"` - generates an accuracy report comparing original and synthetic data
  - `"metrics_only"` - outputs metrics information to stdout without generating an accuracy report
  - `"sample"` - generates a sample report showing distribution comparisons between original data and the data subset used for a training process
  - `"all"` - generates all available reports (*"accuracy"* and *"sample"*)
  
  List example: `["accuracy", "sample"]` to generate multiple report types.

  *Note*: Report generation may require significant time for large tables (>10,000 rows)

- **`fernet_key`** *(Optional[str], default: None)*: The name of the environment variable containing the Fernet key used to decrypt the original data subset. **Important**: Must be the same key used during training if the data was encrypted.

### Required artifacts

To generate quality reports, the following artifacts must exist:

**For accuracy reports (`"accuracy"` or `"metrics_only"`):**
- Training must be completed successfully
- Inference must be completed successfully

**For a `"sample"` report:**
- Training must be completed successfully

### Key notes:

- The method uses existing artifacts and does not re-run a training or an inference process
- All required artifacts must be present in the `model_artifacts` directory
- The `table_name` must match exactly the name of the `table_name` that was used in a training/inference process
- If data was encrypted during a training, the same `fernet_key` must be provided

*Note:* For full documentation and additional details, please refer to [README.md](../README.md)

In [ ]:
# Example 1: Generate an accuracy report after a training and inference completed without a generation of quality reports

# Assume a training and inference were already completed with reports="none"
# Now generate an accuracy report separately
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_conditions",
    reports="accuracy",
    log_level="DEBUG"
)

In [ ]:
launcher_for_multiple_tables.execution_artifacts

In [ ]:
# Example 2: Generate a sample report after training completed

# Generate a sample report to compare original data with its subset
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_properties",
    reports="sample"
)

In [ ]:
# Example 3: Generate multiple reports at once

# Generate both accuracy and sample reports
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_conditions",
    reports=["accuracy", "sample"]
)

# Or use "all" to generate all available reports
launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing_conditions",
    reports="all"
)

In [ ]:
# Example 4: Generate quality reports for encrypted data
import os

# Ensure the Fernet key environment variable is set
# (Should be the same key used during training)
os.environ['MY_FERNET_KEY'] = fernet_key

# Generate qulaity reports with decryption
launcher_for_encrypted_data.generate_quality_reports(
    table_name="housing_encrypted",
    reports="accuracy",
    fernet_key="MY_FERNET_KEY"  # Same key used in training
)

In [ ]:
# Example 5: Generate the "metrics_only" report (without a full accuracy report)
# Output metrics to stdout


launcher_for_multiple_tables.generate_quality_reports(
    table_name="housing",
    reports="metrics_only"
)

# Using Custom Data Loaders

The `loader_path` parameter allows you to load data from non-standard sources such as databases, APIs, cloud storage, or any custom data source. This is particularly useful when:
- Your data is stored in a database (PostgreSQL, MySQL, MongoDB, etc.)
- You need to fetch data from an API or web service
- Your data is in cloud storage (AWS S3, Azure Blob, Google Cloud Storage)
- You need to apply custom preprocessing before training/inference
- The file format is not directly supported

## Custom Loader Requirements

A custom loader function must meet these requirements:

1. **Accept `table_name` as the first parameter** (str): The name of the table to load
2. **Return a pandas DataFrame**: The loaded data in DataFrame format

## Function Signature

```python
def custom_loader(table_name: str) -> pd.DataFrame:
    """
    Load data for the specified table.
    
    Parameters
    ----------
    table_name : str
        The name of the table to load
    
    Returns
    -------
    pd.DataFrame
        The loaded data as a pandas DataFrame
    """
    # Your loading logic here
    return dataframe
```

## Example: Basic Custom Loader

The repository includes a working example in [examples/example-package/custom_loader.py](../examples/example-package/custom_loader.py):

```python
# File: examples/example-package/custom_loader.py
from pathlib import Path
from typing import Optional
import pandas as pd


def get_dataframe(table_name: str) -> pd.DataFrame:
    """
    Load a CSV file as a pandas DataFrame from the example-data directory.
    
    Parameters
    ----------
    table_name : str
        The name of the table (CSV file without extension) to load
    encoding : Optional[str], default="utf-8"
        The encoding to use when reading the CSV file
    
    Returns
    -------
    pd.DataFrame
        The loaded data as a pandas DataFrame
    """
    current_dir = os.path.dirname(os.path.abspath(__file__))
    path_to_example_data = current_dir / "example-data" / f"{table_name}.csv"
    
    if not path_to_example_data.exists():
        raise FileNotFoundError(f"The CSV file '{table_name}.csv' does not exist")
    
    return pd.read_csv(path_to_example_data)
```

## How to Use the Custom Loader

### Step 1: Ensure the loader module is importable

Make sure your custom loader module is importable.

### Step 2: Use the loader during the initialization of the class Syngen

```python
from syngen.sdk import Syngen

launcher = Syngen(loader=get_dataframe, table_name="housing")

launcher.train(
    epochs=5,
    row_limit=1000,
    batch_size=32
)

launcher.infer(
    size=10000,
    batch_size=5000,
    reports="accuracy"
)
```

## Advanced Examples

### Example: Database Loader

```python
# File: my_loaders/db_loader.py
import pandas as pd
from sqlalchemy import create_engine

def load_from_postgres(table_name: str) -> pd.DataFrame:
    """Load data from PostgreSQL database."""
    engine = create_engine("TEST_CONNECTION_STRING")
    query = f"SELECT * FROM {table_name}"
    return pd.read_sql(query, engine)
```

Usage:
```python
Syngen(loader=load_from_postgres, table_name="test_table")
```

### Example: API Loader

```python
# File: my_loaders/api_loader.py
import pandas as pd
import requests

def load_from_api(table_name: str) -> pd.DataFrame:
    """Load data from REST API."""
    response = requests.get(f"{api_url}/{table_name}")
    data = response.json()
    return pd.DataFrame(data)
```

### Example: Cloud Storage Loader

```python
# File: my_loaders/cloud_loader.py
import pandas as pd
import boto3
from io import StringIO

def load_from_s3(table_name: str) -> pd.DataFrame:
    """Load data from AWS S3."""
    s3_client = boto3.client('s3')
    obj = s3_client.get_object(Bucket=bucket_name, Key=f"{table_name}.csv")
    data = obj['Body'].read().decode('utf-8')
    return pd.read_csv(StringIO(data))
```

# Data loading and saving: DataIO class

The SDK provides the `DataIO` class for loading and saving data in various supported formats with optional encryption and format settings. This class is useful when you need to:
- Load and save data in different file formats (*CSV*, *Avro*, *Excel*, etc.)
- Load and save data with specific format settings
- Work with encrypted data files

## Class initialization

```python
DataIO(
    path: str,                          # required: a path to the data file
    fernet_key: Optional[str] = None,   # optional: a Fernet key for encrypted data
    **kwargs                            # optional: format settings for CSV or Excel tables, or schema for AVRO file
)
```

### Parameters description:

- **`path`** *(str, required)*: the path to the data file to load or save. Supported formats include:
  - CSV files: `.csv`, `.psv`, `.tsv`, `.txt`
  - Avro files: `.avro`
  - Excel files: `.xls`, `.xlsx`

- **`fernet_key`** *(Optional[str], default: None)*: the name of the environment variable containing the Fernet key for encrypted data operations.

- **`**kwargs`**: Optional format settings or/and a schema for reading and writing data. Available parameters depend on the file format:

  **For tables in '.csv', '.psv', '.tsv', '.txt' formats:**
  - `sep` *(str)*: Delimiter to use (e.g., `','`, `';'`, `'\t'`)
  - `quotechar` *(str)*: Character used to denote the start and end of a quoted item (default: `'"'`)
  - `quoting` *(str)*: Quoting behavior - `"all"`, `"minimal"`, `"non-numeric"`, `"none"`
  - `escapechar` *(str)*: Character used to escape other characters
  - `encoding` *(str)*: Encoding to use (e.g., `'utf-8'`, `'latin-1'`)
  - `header` *(Optional[int, List[int], Literal["infer"]])*: Row number(s) containing column labels and marking the start of the data
  - `skiprows` *(Optional[int, List[int]])*: Lines to skip at the start of the file
  - `on_bad_lines` *(Literal["error", "warn", "skip"])*: Action on bad lines - `"error"`, `"warn"`, `"skip"`
  - `engine` *(Optional[Literal["c", "python"]])*: Parser engine - `"c"`, `"python"`
  - `na_values` *(Opional[List[str]])*: Additional strings to recognize as NA/NaN

  **For Excel formats (.xls, .xlsx):**
  - `sheet_name` *(Optional[str, int, List[Union[int, str]])*: Name or index of the sheet to read

### Available methods

`load_data(**kwargs)`

Loads data from the specified file path and returns it as a pandas DataFrame.

`load_schema()`

Returns the original schema of the loaded data, including column names and data types. Available only for data in the *'.avro'* format.

`save_data(df, **kwargs)`

Saves a pandas DataFrame to the specified file path with(without) the configured format settings or schema.

*Note:* For full documentation and additional details, please refer to [README.md](../README.md)

In [ ]:
# The example 1: Load CSV data with default settings

from syngen.sdk import DataIO

data_io = DataIO(path="../examples/example-data/housing.csv")

df = data_io.load_data()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# The example 2: Load CSV data with custom format settings

from syngen.sdk import DataIO

data_io = DataIO(
    path="../examples/example-data/escaped_quoted_table.csv",
    sep=',',           # delimiter
    quotechar='"',     # quote character
    quoting="minimal", # quoting style
    encoding='utf-8',  # encoding
    header=0           # use first row as header
)

df = data_io.load_data()

print(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns")
print(f"\nFirst few rows:")
print(df.head())

In [ ]:
# The example 3: Load data and get schema information

from syngen.sdk import DataIO


data_io = DataIO(path="../examples/example-data/avro_file.avro")

df = data_io.load_data()

schema = data_io.load_schema()

print("Data schema:")
print(schema)

In [ ]:
# The example 4: Save data to a file

import pandas as pd
from syngen.sdk import DataIO


sample_data = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'age': [25, 30, 35, 40, 45],
    'city': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']
})


data_io = DataIO(
    path="../examples/example-data/sample_output.csv",
    sep=',',
    encoding='utf-8'
)


data_io.save_data(sample_data)

print("Data saved successfully to '../examples/example-data/sample_output.csv'")